<a href="https://colab.research.google.com/github/ysooch0819/AI16-Projects/blob/main/S3Project_flask_app.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# flask_app/submit_test.py
from flask import Flask, render_template, request
import pickle
import requests
import pandas as pd
import os
import warnings
warnings.filterwarnings('ignore')
from dotenv import load_dotenv
load_dotenv()
riot_api_key = os.environ.get('riot_api_key')

app = Flask(__name__, static_url_path='/static')

## GET 방식으로 값을 전달받음. 
## num이라는 이름을 가진 integer variable를 넘겨받는다고 생각하면 됨. 
## 아무 값도 넘겨받지 않는 경우도 있으므로 비어 있는 url도 함께 mapping해주는 것이 필요함
@app.route('/')
def main_get(num=None):
    return render_template('submit_test.html', num=num)

@app.route('/nickname', methods=['POST', 'GET'])
def calculate(num=None):
    ## 어떤 http method를 이용해서 전달받았는지를 아는 것이 필요함
    ## 아래에서 보는 바와 같이 어떤 방식으로 넘어왔느냐에 따라서 읽어들이는 방식이 달라짐
    if request.method == 'POST':
        #temp = request.form['num']
        pass
    elif request.method == 'GET':
        ## 넘겨받은 문자
        nickname = request.args.get('summonerName')

        # X_test 데이터 만들기
        url_for_puuid = f'https://kr.api.riotgames.com/tft/summoner/v1/summoners/by-name/{nickname}?api_key={riot_api_key}'
        puuid = requests.get(url_for_puuid).json()['puuid']
        url_for_1_match = f"https://asia.api.riotgames.com/tft/match/v1/matches/by-puuid/{puuid}/ids?start=0&count=1&api_key={riot_api_key}"
        url_for_match_detail = f"https://asia.api.riotgames.com/tft/match/v1/matches/{requests.get(url_for_1_match).json()[0]}?api_key={riot_api_key}"
        matchdata = pd.json_normalize(requests.get(url_for_match_detail).json()['info']['participants'])

        # Data Wrangling 용 함수
        def unitCost(argument):
            switcher = {
                0: 1,
                1: 2,
                2: 3,
                3: 6,
                4: 4,
                5: 7,
                6: 5,
                7: 10,
            }
            return switcher.get(argument, -1)

        user_data = matchdata[matchdata['puuid'] == puuid].reset_index().drop(columns='index')

        tmp = user_data['units'].iloc[0]
        result_list =[]
        cols = ['cost_sum', 'highCost_sum', 'highCost_ratio']
        units_df = pd.DataFrame(columns = cols)
        for idx in range(len(tmp)):
            cost_sum = 0
            high_cost_sum = 0
            for idx in range(0, len(tmp)):
                cost = unitCost(tmp[idx]['rarity'])
                cost_sum += cost*(3**(tmp[idx]['tier']-1))
                if cost >=4: # 4코스트 이상의 유닛에 사용한 총 골드
                    high_cost_sum += cost*(3**(tmp[idx]['tier']-1))

            high_cost_ratio = high_cost_sum / cost_sum
        
        result_list += cost_sum, high_cost_sum, high_cost_ratio
        units_df = units_df.append(pd.DataFrame([result_list], columns=cols))

        # 테스트 데이터 만들기
        X_test = user_data.merge(units_df, left_index=True, right_index=True, how='left')[['level', 'cost_sum', 'highCost_sum', 'highCost_ratio']]
        
        # 테스트의 실제 결과
        y_real = user_data['placement'].iloc[0]
        
        # 피클링된 모델 가져오기
        model = None
        with open('model/model.pkl', 'rb') as pickle_file:
             model = pickle.load(pickle_file)
        y_pred = model.predict(X_test)[0]
        ## 넘겨받은 값을 원래 페이지로 리다이렉트
        return render_template('submit_test.html', summonerName=nickname, y_pred = int(y_pred), y_true = int(y_real))
    ## else 로 하지 않은 것은 POST, GET 이외에 다른 method로 넘어왔을 때를 구분하기 위함

if __name__ == '__main__':
    # threaded=True 로 넘기면 multiple plot이 가능해짐
  app.run(debug=True, threaded=True)

In [ ]:
# flask_app/templates/submit_test.html
<!DOCTYPE html>
<html>
<head>
    <title>
        롤토체스-전략적팀전투 순위 예측
    </title>
    <style>
        form {
            /* Just to center the form on the page */
            margin: 0 auto;
            width: 400px;
            /* To see the outline of the form */
            padding: 1em;
            border: 1px solid #CCC;
            border-radius: 1em;
        }
        form div + div {
            /* div div 사이에 여백 추가*/
            margin-top: 1em;
        }
        label {
            /* To make sure that all label have the same size and are properly align */
            display: inline-block;
            width: 90px;
            text-align: right;
        }
        .button {
            /* To position the buttons to the same position of the text fields */
            padding-left: 90px; /* same size as the label elements */
        }

        button {
            /* This extra margin represent roughly the same space as the space
            between the labels and their text fields */
            margin-left: .5em;
        }
    </style>
</head>
<body>
    <h1><center> 소환사 이름을 입력하세요.</center></h1><br>
    <form method="GET" action="/nickname">
        <div>
            <label for="summonerName"> 아이디 </label>
            <input type="text" name="summonerName">
        </div>
        <div class='button'>
            <button type="submit">데이터 불러오기</button>
        </div>
    </form>
    <div>
        <p>
            {% if summonerName == None %}
                <h5> 잘못된 입력입니다. 소환사명을 확인해주세요. </h5>
            {% else %}
                <h3> {{summonerName}}님의 최근 1경기 예상 순위는 {{y_pred}} 위 입니다.</h3>
                <h3> {{summonerName}}님의 최근 1경기 실제 순위는 {{y_true}} 위 입니다.</h3>
            {% endif %}
            {% if y_pred == y_true%}
                <h4> 실력을 제대로 보여주었군요! </h4>
            {% elif y_pred < y_true%}
                <h4> 운이 없었네요. 다음 경기는 행운이 함께하길! </h4>
            {% else %}
                <h4> 축하드립니다! 운이 좋았네요!! </h4>
            {% endif %}
        </p>
    </div>
</body>
</html>